In [1]:
import pickle 
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdmolfiles import SDMolSupplier

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [62]:
from collections import defaultdict
from rdkit import Chem
from rdkit.Chem.rdmolfiles import SDMolSupplier

def getEleSpread(smiList):
    element_mol = defaultdict(list)
    element = set()
    for smi in smiList:
        #mol = smi
        mol = Chem.MolFromSmiles(smi)
        if mol:
            for c in set([atom.GetSymbol() for atom in mol.GetAtoms()]):
                #if c not in element_mol:
                    element_mol[c].append(smi)
                #element.add(c)
            #break
    return {i:len(item) for i, item in element_mol.items()}

In [24]:
from rdkit.Chem.rdmolops import GetFormalCharge

def getCharge(inchi):
    mol = Chem.MolFromInchi(inchi)
    return GetFormalCharge(mol)

In [29]:
from rdkit import Chem

def getinchi(x):
    try:
        mol = Chem.MolFromSmiles(x)
        return Chem.MolToInchi(mol)
    except:
        return None

In [2]:
train = pd.read_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/train.csv')
valid = pd.read_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/valid.csv')
test = pd.read_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/test.csv')


In [3]:
train.shape, valid.shape, test.shape

((2430, 211), (270, 211), (301, 211))

In [4]:
train.head()

,InChI,SMILES,target,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,InChI=1S/C7H4Cl2O3/c8-3-1-4(7(11)12)6(10)5(9)2...,c1c(cc(c(c1C(=O)O)O)Cl)Cl,224.0,10.448920,-1.266759,10.448920,0.069537,0.742606,207.012,202.980,...,0,0,0,0,0,0,0,0,0,0
1,InChI=1S/C8H4N2O2/c11-5-9-7-1-2-8(4-3-7)10-6-1...,O=C=Nc1ccc(cc1)N=C=O,94.0,9.807455,0.481342,9.807455,0.481342,0.487130,160.132,156.100,...,0,0,0,0,0,0,0,0,0,0
2,"InChI=1S/C9H12/c1-3-9-6-4-8(2)5-7-9/h4-7H,3H2,...",CCc1ccc(cc1)C,-62.0,2.175926,1.139167,2.175926,1.139167,0.533979,120.195,108.099,...,0,0,0,0,0,0,0,0,0,0
3,"InChI=1S/CH4N2O/c2-3-1-4/h1H,2H2,(H,3,4)",C(=O)NN,57.0,7.458333,0.527778,7.458333,0.527778,0.170361,60.056,56.024,...,0,0,0,0,0,0,0,0,0,0
4,InChI=1S/C14H9Cl5/c15-10-7-5-9(6-8-10)13(14(17...,C1=CC=C(C(=C1)C(C2=CC=C(C=C2)Cl)C(Cl)(Cl)Cl)Cl,75.0,6.204208,-1.501168,6.204208,0.455123,0.540334,354.491,345.419,...,0,0,0,0,0,0,0,0,0,0


In [5]:
all_ = pd.concat([train, valid, test])

In [6]:
all_.shape

(3001, 211)

In [26]:
train_old = pd.read_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/old/train.csv', names=['SMILES', 'target'])
valid_old = pd.read_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/old/valid.csv', names=['SMILES', 'target'])
test_old = pd.read_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/old/test.csv', names=['SMILES', 'target'])


In [27]:
train_old.shape, valid_old.shape, test_old.shape

((2462, 2), (274, 2), (305, 2))

In [30]:
train_old['InChI'] = train_old['SMILES'].apply(lambda x: getinchi(x))
valid_old['InChI'] = valid_old['SMILES'].apply(lambda x: getinchi(x))
test_old['InChI'] = test_old['SMILES'].apply(lambda x: getinchi(x))


In [35]:
train_old[pd.isnull(train_old['InChI'])].shape

(14, 3)

In [36]:
valid_old[pd.isnull(valid_old['InChI'])].shape

(1, 3)

In [37]:
test_old[pd.isnull(test_old['InChI'])].shape

(1, 3)

In [31]:
train_old.head()

,SMILES,target,InChI
0,Cc1cc(ccc1N)c2ccc(c(c2)C)N,130.0,InChI=1S/C14H16N2/c1-9-7-11(3-5-13(9)15)12-4-6...
1,Cc1cc(O)ccc1,11.0,"InChI=1S/C7H8O/c1-6-3-2-4-7(8)5-6/h2-5,8H,1H3"
2,COc1ccc(cc1C(=O)NCCc2ccc(cc2)S(=O)(=O)NC(=O)NC...,173.0,InChI=1S/C23H28ClN3O5S/c1-32-21-12-9-17(24)15-...
3,COC1=CC=CC(=C1)C(=O)O,106.0,InChI=1S/C8H8O3/c1-11-7-4-2-3-6(5-7)8(9)10/h2-...
4,Clc1ccc(Cl)c(N)c1,48.0,"InChI=1S/C6H5Cl2N/c7-4-1-2-5(8)6(9)3-4/h1-3H,9H2"


In [49]:
all_old = pd.concat([train_old, valid_old, test_old]).reset_index(drop=True)

In [50]:
all_old.shape

(3041, 3)

In [63]:
getEleSpread(all_old['SMILES'])

{'C': 3016,
 'N': 1203,
 'O': 2000,
 'S': 255,
 'Cl': 462,
 'Br': 210,
 'I': 58,
 'P': 30,
 'F': 162,
 'Si': 19,
 'B': 5}

In [76]:
import glob
mols = []
names1 = []
i = 0
for file in glob.glob('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/sdf/*.sdf'):
    mols.append(SDMolSupplier(file)[0])
    names1.append(file.split('/')[-1].split('.')[0])
    i += 1

In [23]:
getEleSpread(mols)

{'C': 3010,
 'O': 1994,
 'N': 1202,
 'F': 162,
 'S': 255,
 'Br': 209,
 'I': 58,
 'P': 30,
 'Cl': 462,
 'Si': 19}

In [93]:
len(mols), len(names)

(0, 3018)

In [53]:
issdf = []
for i in range(all_old.shape[0]):
    if str(i) in names: issdf.append(True)
    else: issdf.append(False)

In [54]:
all_old['issdf'] = issdf

In [100]:
all_old_use = all_old[(~pd.isnull(all_old['InChI'])) & (all_old['issdf'])].reset_index(drop=True)

In [101]:
all_old_use.shape

(3018, 4)

In [81]:
import glob
mols = []
names = []
i = 0
for file in glob.glob('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/MMFFXYZ/*.xyz'):
    #mols.append(SDMolSupplier(file)[0])
    names.append(file.split('/')[-1].split('.')[0])
    i += 1

In [121]:
k = 0
inchi_idx = {}
for i,inchi in enumerate(all_old['InChI']):
    if not inchi: continue
    if str(i) in names:
        k += 1
        inchi_idx[inchi] = i

In [122]:
len(inchi_idx)

3015

In [99]:
k

3018

In [87]:
all_old[(~pd.isnull(all_old['InChI']))].shape

(3025, 4)

There are 3 duplicates. 

In [90]:
inchi_idx['InChI=1S/C5H8/c1-3-5-4-2/h3-5H,1H2,2H3']

2570

In [102]:
all_old_use[all_old_use.duplicated('InChI', False)]

,SMILES,target,InChI,issdf
670,C=CC=C/C,-141.0,"InChI=1S/C5H8/c1-3-5-4-2/h3-5H,1H2,2H3",True
943,O=C(OCC)c1ccc(O)cc1,116.0,InChI=1S/C9H10O3/c1-2-12-9(11)7-3-5-8(10)6-4-7...,True
1202,CC=C(C)CC,-135.0,"InChI=1S/C6H12/c1-4-6(3)5-2/h4H,5H2,1-3H3",True
1296,CC=C(/C)CC,-138.0,"InChI=1S/C6H12/c1-4-6(3)5-2/h4H,5H2,1-3H3",True
2549,C=CC=CC,-87.0,"InChI=1S/C5H8/c1-3-5-4-2/h3-5H,1H2,2H3",True
2837,CCOC(=O)C1=CC=C(C=C1)O,116.0,InChI=1S/C9H10O3/c1-2-12-9(11)7-3-5-8(10)6-4-7...,True


In [104]:
len(pickle.load(open('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/inchi_index.pt', 'rb')))

3015

In [106]:
all_old_use_group = all_old_use.groupby('InChI').mean()

In [107]:
all_old_use_group['InChI'] = all_old_use_group.index

In [108]:
all_old_use_group

,target,issdf,InChI
InChI,,,
InChI=1S/Br2/c1-2,-7.2,True,InChI=1S/Br2/c1-2
InChI=1S/C10F8/c11-3-1-2(5(13)9(17)7(3)15)6(14)10(18)8(16)4(1)12,87.0,True,InChI=1S/C10F8/c11-3-1-2(5(13)9(17)7(3)15)6(14...
"InChI=1S/C10H10N2/c1-2-4-10(5-3-1)8-12-7-6-11-9-12/h1-7,9H,8H2",71.0,True,InChI=1S/C10H10N2/c1-2-4-10(5-3-1)8-12-7-6-11-...
"InChI=1S/C10H10N2/c1-7-6-9(11)8-4-2-3-5-10(8)12-7/h2-6H,1H3,(H2,11,12)",164.0,True,InChI=1S/C10H10N2/c1-7-6-9(11)8-4-2-3-5-10(8)1...
"InChI=1S/C10H10N2/c11-8-5-1-3-7-4-2-6-9(12)10(7)8/h1-6H,11-12H2",63.0,True,InChI=1S/C10H10N2/c11-8-5-1-3-7-4-2-6-9(12)10(...
...,...,...,...
InChI=1S/Cl3HSi/c1-4(2)3/h4H,-127.0,True,InChI=1S/Cl3HSi/c1-4(2)3/h4H
"InChI=1S/F2O2S/c1-5(2,3)4",-135.8,True,"InChI=1S/F2O2S/c1-5(2,3)4"
InChI=1S/I2/c1-2,113.5,True,InChI=1S/I2/c1-2


In [109]:
all_old_use_group['SMILES'] = all_old_use_group['InChI'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromInchi(x)))

In [110]:
from sklearn.model_selection import train_test_split

In [111]:
rest, test = train_test_split(all_old_use_group, test_size=0.1, random_state=123)
train, valid = train_test_split(rest, test_size=0.1, random_state=123)

In [112]:
train.shape, valid.shape, test.shape

((2441, 4), (272, 4), (302, 4))

In [113]:
train.head()

,target,issdf,InChI,SMILES
InChI,,,,
"InChI=1S/C8H18/c1-5-7-8(3,4)6-2/h5-7H2,1-4H3",-126.0,True,"InChI=1S/C8H18/c1-5-7-8(3,4)6-2/h5-7H2,1-4H3",CCCC(C)(C)CC
"InChI=1S/C3H5NS2/c5-3-4-1-2-6-3/h1-2H2,(H,4,5)",105.0,True,"InChI=1S/C3H5NS2/c5-3-4-1-2-6-3/h1-2H2,(H,4,5)",SC1=NCCS1
"InChI=1S/C8H9Cl/c1-7-4-2-3-5-8(7)6-9/h2-5H,6H2,1H3",-2.0,True,"InChI=1S/C8H9Cl/c1-7-4-2-3-5-8(7)6-9/h2-5H,6H2...",Cc1ccccc1CCl
"InChI=1S/C7H7NO/c9-8-6-7-4-2-1-3-5-7/h1-6,9H",36.5,True,"InChI=1S/C7H7NO/c9-8-6-7-4-2-1-3-5-7/h1-6,9H",ON=Cc1ccccc1
"InChI=1S/C14H12O2/c15-14(16)13(11-7-3-1-4-8-11)12-9-5-2-6-10-12/h1-10,13H,(H,15,16)",147.0,True,InChI=1S/C14H12O2/c15-14(16)13(11-7-3-1-4-8-11...,O=C(O)C(c1ccccc1)c1ccccc1


In [115]:
print(getEleSpread(all_old_use_group['SMILES']).keys())

dict_keys(['Br', 'C', 'F', 'N', 'O', 'Cl', 'S', 'P', 'I', 'Si'])


In [118]:
all_old_use_group['charge'] = all_old_use_group['InChI'].apply(lambda x: getCharge(x))

In [119]:
all_old_use_group.describe()

,target,charge
count,3015.000000,3015.0
mean,62.364295,0.0
std,95.513292,0.0
min,-188.000000,0.0
25%,5.000000,0.0
50%,63.000000,0.0
75%,129.000000,0.0
max,438.000000,0.0


In [120]:
train[['InChI', 'SMILES', 'target']].to_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/train.csv', index=False)
valid[['InChI', 'SMILES', 'target']].to_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/valid.csv', index=False)
test[['InChI', 'SMILES', 'target']].to_csv('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/test.csv', index=False)



In [123]:
with open('/scratch/dz1061/gcn/chemGraph/data/mp/bradley/split/inchi_index.pt', 'wb') as f:
    pickle.dump(inchi_idx, f, protocol=pickle.HIGHEST_PROTOCOL)